In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import seaborn as sns
import missingno as msno
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
train=pd.read_csv('./datasets/train.csv')

In [4]:
test=pd.read_csv('./datasets/test.csv')

#### Set the index to 'Id' for both dataframes

In [5]:
train.set_index('Id', inplace = True)
test.set_index('Id', inplace = True)

In [6]:
train.loc[train['Garage Cars']>4]

,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
747,903236040,30,RM,50.0,8635,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,8,2009,WD,126500


#### dropping this row because it says this 1k sqft house has a 5 car garage which is insane

In [7]:
train.drop(747,axis=0,inplace=True)

#### dropping rows with 3 or more fireplaces because they are outliers only 7 of them

In [10]:
train.loc[train['Fireplaces']>3]

,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
2499,532476050,80,RL,100.0,14330,Pave,NaN,IR1,Low,AllPub,...,0,800,Gd,GdPrv,NaN,0,1,2006,WD,260000


In [13]:
train.loc[train['Fireplaces']>2]

,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1499,908154235,60,RL,313.0,63887,Pave,NaN,IR3,Bnk,AllPub,...,0,480,Gd,NaN,NaN,0,1,2008,New,160000
786,905401100,20,RL,NaN,10708,Pave,NaN,IR1,Lvl,AllPub,...,142,0,NaN,GdWo,NaN,0,11,2009,COD,190000
344,526355080,80,RL,75.0,13860,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,7,2009,WD,345000
627,535125090,60,RL,85.0,13600,Pave,NaN,Reg,Lvl,AllPub,...,189,0,NaN,NaN,NaN,0,10,2009,WD,205000
1925,535177110,20,RL,80.0,9736,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2007,WD,174850
2501,533125120,60,RL,105.0,11025,Pave,NaN,Reg,HLS,AllPub,...,0,0,NaN,NaN,NaN,0,10,2006,WD,334000


In [14]:
train.drop(1499,axis=0,inplace=True)

In [15]:
train.drop(786,axis=0,inplace=True)

In [16]:
train.drop(344,axis=0,inplace=True)

In [17]:
train.drop(627,axis=0,inplace=True)

In [18]:
train.drop(1925,axis=0,inplace=True)

In [19]:
train.drop(2501,axis=0,inplace=True)

In [22]:
train.isnull().sum().sort_values()

PID                0
Heating QC         0
Central Air        0
Electrical         0
1st Flr SF         0
                ... 
Fireplace Qu     999
Fence           1646
Alley           1903
Misc Feature    1978
Pool QC         2036
Length: 80, dtype: int64

#### For the columns that had only 1-2 nulls, I replaced them with zeros

In [23]:
train['Overall Cond'].value_counts()

5    1164
6     366
7     269
8     101
4      69
3      35
9      29
2       6
1       4
Name: Overall Cond, dtype: int64

In [24]:
train['Garage Area'].replace(np.nan,0, inplace=True)

In [25]:
test['Garage Area'].replace(np.nan,0, inplace=True)

In [26]:
train['BsmtFin SF 2'].replace(np.nan,0, inplace=True)

In [27]:
test['BsmtFin SF 2'].replace(np.nan,0, inplace=True)

In [28]:
train['Garage Cars'].replace(np.nan,0, inplace=True)

In [29]:
test['Garage Cars'].replace(np.nan,0, inplace=True)

In [30]:
train['Bsmt Half Bath'].replace(np.nan,0, inplace=True)

In [31]:
test['Bsmt Half Bath'].replace(np.nan,0, inplace=True)

In [32]:
train['Bsmt Unf SF'].replace(np.nan,0, inplace=True)

In [33]:
test['Bsmt Unf SF'].replace(np.nan,0, inplace=True)

In [34]:
train['Bsmt Full Bath'].replace(np.nan,0, inplace=True)

In [35]:
test['Bsmt Full Bath'].replace(np.nan,0, inplace=True)

In [36]:
test['Total Bsmt SF'].replace(np.nan,0, inplace=True)

In [37]:
train['Total Bsmt SF'].replace(np.nan,0, inplace=True)

#### Get dummies for Neighborhood to be included in the model

In [38]:
train_dummies = pd.get_dummies(train['Neighborhood'])
test_dummies  = pd.get_dummies(test['Neighborhood'])

In [39]:
#finding the columns that are in one but not the other
missing_columns_train = list(set(test_dummies) - set(train_dummies))

missing_columns_test  = list(set(train_dummies) - set(test_dummies))

In [40]:
missing_columns_train

[]

In [41]:
missing_columns_test

['Landmrk', 'GrnHill']

In [42]:
for col in missing_columns_test:
    test_dummies[col] = 0

In [43]:
train_dummies.columns

Index(['Blmngtn', 'Blueste', 'BrDale', 'BrkSide', 'ClearCr', 'CollgCr',
       'Crawfor', 'Edwards', 'Gilbert', 'Greens', 'GrnHill', 'IDOTRR',
       'Landmrk', 'MeadowV', 'Mitchel', 'NAmes', 'NPkVill', 'NWAmes',
       'NoRidge', 'NridgHt', 'OldTown', 'SWISU', 'Sawyer', 'SawyerW',
       'Somerst', 'StoneBr', 'Timber', 'Veenker'],
      dtype='object')

In [44]:
test_dummies.columns

Index(['Blmngtn', 'Blueste', 'BrDale', 'BrkSide', 'ClearCr', 'CollgCr',
       'Crawfor', 'Edwards', 'Gilbert', 'Greens', 'IDOTRR', 'MeadowV',
       'Mitchel', 'NAmes', 'NPkVill', 'NWAmes', 'NoRidge', 'NridgHt',
       'OldTown', 'SWISU', 'Sawyer', 'SawyerW', 'Somerst', 'StoneBr', 'Timber',
       'Veenker', 'Landmrk', 'GrnHill'],
      dtype='object')

In [45]:
#drop first
train_dummies.drop(columns = ['Blmngtn'], inplace = True)
test_dummies.drop(columns = ['Blmngtn'], inplace = True)

#### Create a new dataframe by taking the old one and merging it with the dummy columns

In [46]:
#from Noah's in class sample project
trainall = train.join(train_dummies)
testall  = test.join(test_dummies)

## Make sure the columns in `df_test_full` are in the same order as the column in `df_train_full`
testall = testall[trainall.drop(columns = ['SalePrice']).columns]

In [47]:
trainall.columns

Index(['PID', 'MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area', 'Street',
       'Alley', 'Lot Shape', 'Land Contour', 'Utilities',
       ...
       'NoRidge', 'NridgHt', 'OldTown', 'SWISU', 'Sawyer', 'SawyerW',
       'Somerst', 'StoneBr', 'Timber', 'Veenker'],
      dtype='object', length=107)

In [48]:
testall.columns

Index(['PID', 'MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area', 'Street',
       'Alley', 'Lot Shape', 'Land Contour', 'Utilities',
       ...
       'NoRidge', 'NridgHt', 'OldTown', 'SWISU', 'Sawyer', 'SawyerW',
       'Somerst', 'StoneBr', 'Timber', 'Veenker'],
      dtype='object', length=106)

#### Combine 1st and 2nd floor square footage to get total square footage

In [49]:
#combine 1st and 2nd floor sq ft
trainall['sq_ft']=trainall['1st Flr SF']+trainall['2nd Flr SF']
testall['sq_ft']=testall['1st Flr SF']+testall['2nd Flr SF']

In [50]:
trainall['sq_ft'].value_counts()

864     25
1040    20
1092    19
1456    15
1200    14
        ..
1122     1
1767     1
1770     1
1771     1
1265     1
Name: sq_ft, Length: 1047, dtype: int64

In [51]:
trainall.loc[trainall['sq_ft']>4500]

,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,...,NridgHt,OldTown,SWISU,Sawyer,SawyerW,Somerst,StoneBr,Timber,Veenker,sq_ft
Id,,,,,,,,,,,,,,,,,,,,,
2181,908154195,20,RL,128.0,39290,Pave,NaN,IR1,Bnk,AllPub,...,0,0,0,0,0,0,0,0,0,5095


#### This value for total square feet is an outlier because the square footage is way to high for the price

In [52]:
trainall.drop(2181,axis=0,inplace=True)

#### Create a new column called age which is 2020 minus the year the house was built

In [54]:
trainall['age']=2020-trainall['Year Built']

In [55]:
testall['age']=2020-testall['Year Built']

#### Get dummies for Kitchen Quality to be included in the model

In [56]:
trainall_dummies2=pd.get_dummies(trainall['Kitchen Qual'],drop_first=True)
trainall_dummies2

,Fa,Gd,TA
Id,,,
109,0,1,0
544,0,1,0
153,0,1,0
318,0,0,1
255,0,0,1
...,...,...,...
1587,0,1,0
785,0,0,1
916,0,0,1


In [57]:
testall_dummies2=pd.get_dummies(testall['Kitchen Qual'],drop_first=True)
testall_dummies2

,Fa,Gd,Po,TA
Id,,,,
2658,1,0,0,0
2718,0,0,0,1
2414,0,1,0,0
1989,0,0,0,1
625,0,0,0,1
...,...,...,...,...
1662,0,0,0,1
1234,0,1,0,0
1373,0,0,0,1


#### Drop 'Po' column from test dataframe since train does not have that one

In [58]:
testall_dummies2.drop(columns=['Po'],inplace=True)

#### Merge the new dummy columns with our dataframe

In [59]:
trainall = trainall.join(trainall_dummies2)
testall  = testall.join(testall_dummies2)

## Make sure the columns are in the same order 
testall = testall[trainall.drop(columns = ['SalePrice']).columns]

#### Get dummies for Street to be included in the model

In [60]:
trainall_dummies3=pd.get_dummies(trainall['Street'],drop_first=True)

In [61]:
testall_dummies3=pd.get_dummies(testall['Street'],drop_first=True)

In [62]:
testall_dummies3

,Pave
Id,
2658,1
2718,1
2414,1
1989,1
625,1
...,...
1662,1
1234,1
1373,1


#### Merge the new dummy columns with our dataframe

In [63]:
trainall = trainall.join(trainall_dummies3)
testall  = testall.join(testall_dummies3)

## Make sure the columns are in the same order 
testall = testall[trainall.drop(columns = ['SalePrice']).columns]

#### Get dummies for Finished Basement Type 1 to be included in the model

In [64]:
trainall_dummies4=pd.get_dummies(trainall['BsmtFin Type 1'],drop_first=True)

In [65]:
testall_dummies4=pd.get_dummies(testall['BsmtFin Type 1'],drop_first=True)

#### Merge the new dummy columns with our dataframe

In [66]:
trainall = trainall.join(trainall_dummies4)
testall  = testall.join(testall_dummies4)


#### Get dummies for Lot Shape to be included in the model

In [67]:
trainall_dummies5=pd.get_dummies(trainall['Lot Shape'],drop_first=True)

In [68]:
testall_dummies5=pd.get_dummies(testall['Lot Shape'],drop_first=True)

#### Merge the new dummy columns with our dataframe

In [69]:
trainall = trainall.join(trainall_dummies5)
testall  = testall.join(testall_dummies5)


#### Get dummies for Exterior Covering to be included in the model

In [70]:
trainall_dummies6=pd.get_dummies(trainall['Exterior 1st'],drop_first=True)

In [71]:
testall_dummies6=pd.get_dummies(testall['Exterior 1st'],drop_first=True)

#### Merge the new dummy columns with our dataframe

In [72]:
trainall = trainall.join(trainall_dummies6)
testall  = testall.join(testall_dummies6)


In [73]:
trainall.head()

,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,...,CemntBd,HdBoard,ImStucc,MetalSd,Plywood,Stone,Stucco,VinylSd,Wd Sdng,WdShing
Id,,,,,,,,,,,,,,,,,,,,,
109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,AllPub,...,0,1,0,0,0,0,0,0,0,0
544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,0,0,0,1,0,0
153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,0,0,0,0,1,0,0
318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,0,0,0,0,1,0,0
255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,0,0,0,0,0,1,0


#### Export our dataframes to be continued on the next notebook

In [75]:
trainall.to_csv('./datasets/trainall_v6.csv',index=False)


In [74]:
testall.to_csv('./datasets/testall_v6.csv',index=False)

# Continue to Modeling Notebook